In [11]:
import time 
import json

ar=['/content/drive/MyDrive/CS634/db1_transactions.json','/content/drive/MyDrive/CS634/db2_transactions.json','/content/drive/MyDrive/CS634/db3_transactions.json','/content/drive/MyDrive/CS634/db4_transactions.json','/content/drive/MyDrive/CS634/db5_transactions.json']
items = json.loads(open('/content/drive/MyDrive/CS634/items.json','r').read())
print(items)

['google pixel 5',
 'google pixel 5 case',
 'google pixel 5 screen protector',
 'lg gram 13',
 'logitech g502',
 'sony wh-1000xm4',
 'bose quietcomfort 35 ii',
 'corsair k96 platinum',
 'asus flow x13',
 'sennheiser hd 600',
 'samsung galaxy buds',
 'sony wf-1000xm3',
 'anker wireless charger',
 'iphone 12',
 'apple airpods',
 'iphone 12 case',
 'macbook pro',
 'rocketbook core',
 'apple ipad',
 'microsoft surface',
 'logitech g915',
 'corsair scimitar pro',
 'samsung galaxy s21',
 'samsung galaxy s21 case',
 'usb c charger',
 'thunderbolt dongle',
 'lenovo x1c',
 'rog strix g15',
 'razer kraken',
 'laptop case']

In [ ]:
def emptyItemSetsList(n): 
  rtn = []
  for i in range(n):
    rtn.append([])
  return rtn
def subsets(s): # returns all subsets
  sets = []
  for i in range(1 << len(s)):
    subset = [s[bit] for bit in range(len(s)) if (i & (1 << bit) > 0)] 
    sets.append(subset) 
  return sets
def getConfidence(itemsetLeft, itemsetRight, transactions, sup_vals):
  return getSupport(itemsetLeft.union(itemsetRight), transactions, sup_vals)/getSupport(itemsetLeft, transactions, sup_vals)
def getSupport(itemset, transactions, sup_vals): 
  if tuple(sorted(list(itemset))) not in sup_vals:
    lstOfOccurances = list()
    occurances_of_items_together = set()
    for itm in itemset:
      if itm not in transactions:
        return 0.0
      lstOfOccurances.append(set(transactions[itm]))
    finalSet = set()
    finalSet = finalSet.union(lstOfOccurances[0])
    for x in range(1, len(lstOfOccurances)):
      finalSet = finalSet.intersection(lstOfOccurances[x])
    sup_vals[tuple(list(sorted(list(itemset))))] = len(finalSet)/len(items)
  return sup_vals[tuple(list(sorted(list(itemset))))]
def printRules(rules): 
  for rule in rules:
    print(rule[0], " -> ", rule[1], " : ", rule[2], " ,", rule[3])
def idk(db): #takes all transaction: key->item value->list of item locations
  items_transactionIDS = dict() 
  for i, transaction in enumerate(db): 
    for it in transaction: 
      if it not in items_transactionIDS: 
        items_transactionIDS[it] = [] 
      items_transactionIDS[it].append(i)
  return(items_transactionIDS)
def itemGen(n): # generates k-itemsets 
  itms=set()
  for x in items:
    itms.add(x)
  for i in range(n-1):
    iii=set()
    for j in itms:
      for k in items:
        if k in j:
          continue
        else:
          x=[]
          if type(j)==str:
            x.append(j)
          else:
            for y in j:
              x.append(y)
          x.append(k)
          iii.add(tuple(sorted(x)))
    itms=iii
  return(itms)
def pp(lst): #returns all combinations of list
    if len(lst) == 0: 
        return [] 
    if len(lst) == 1: 
        return [lst] 
    l = []  
    for i in range(len(lst)): 
       m = lst[i] 
       remLst = lst[:i] + lst[i+1:] 
       for p in pp(remLst): 
           l.append([m] + p) 
    return l 
def bruteForce(db, minS, minC): 
  supercool=True
  k=1
  ddbb=idk(db)
  while(supercool): #keep going if calculated support val is higher than input
    k+=1
    supercool=False
    itms=itemGen(k)
    for i in itms:
      sup_vals=dict()
      supp = getSupport(tuple(i),ddbb,sup_vals)
      if supp>=minS:
        supercool=True
        perm=pp(list(i))
        for w in perm: #go through all combinations
          for q in range(len(w)):
            left,right=set(),set()
            left.update(w[:q])
            right.update(w[q:])
            sup_vals=dict()
            if len(left)==0 or len(right)==0:
              continue
            conf=getConfidence(left, right, ddbb,sup_vals)
            if conf >= minC:
              print(left, " -> ", right, ": [", supp, ", ", conf, "]")
def apriori(db, minS, minC):
  db = json.loads(open(db,'r').read())  
  items_transactionIDS=idk(db)
  freqItemSets = emptyItemSetsList(len(items)) 
  sup_vals = dict() 
  for item in items_transactionIDS: #generate 1-itemsets 
    itemset = set() 
    itemset.add(item) 
    supp = getSupport(itemset, items_transactionIDS, sup_vals) 
    if supp >= minS: 
      freqItemSets[0].append(itemset) 
  for k in range(1, len(items)): #generate k-itemsets 
    prev_itemsets = freqItemSets[k-1] 
    for i in range(len(prev_itemsets)): 
      for j in range(len(prev_itemsets)): 
        if i == j: 
          continue 
        set1 = prev_itemsets[i]
        set2 = prev_itemsets[j] 
        set3 = set1.union(set2) 
        if len(set3) == k+1 and set3 not in freqItemSets[k] and getSupport(set3, items_transactionIDS, sup_vals) >= minS: 
          freqItemSets[k].append(set3)  
  rules = set() 
  for itemsets in freqItemSets: #find confidence and stuff
    if len(itemsets) == 0: 
      break 
    for itemset in itemsets: 
      if len(itemset) <= 1: 
        continue 
      orderedTup = tuple(sorted(list(itemset))) 
      allSubsets = subsets(list(orderedTup)) 
      for x in allSubsets: 
        itemsetLeft = set(orderedTup).difference(set(x)) 
        itemsetRight = set(x) 
        if len(itemsetLeft) == 0 or len(itemsetRight) == 0: 
          continue 
        conf = getConfidence(set(itemsetLeft), set( itemsetRight), items_transactionIDS, sup_vals) 
        if conf >= minC: 
          rule = [tuple(sorted(itemsetLeft)), tuple(sorted(itemsetRight)), getSupport( itemset, items_transactionIDS, sup_vals), conf] 
          rules.add(tuple(rule)) 
  printRules(sorted(list(rules))) 

In [ ]:
minSupport = float(input("Enter minimum support (out of 100): "))
minConfidence = float(input("Enter minimum confidence (out of 100): "))
print('brute force timer start')
timeBruteForceStart = time.time()

for qq in ar:
  print("db: ", qq)
  bruteForce(json.loads(open(qq,'r').read()) , minSupport/100.0, minConfidence/100.0)
  print("\n\n")

timeBruteForceEnd = time.time()
print("brute Force time for all 5 databases: ", timeBruteForceEnd - timeBruteForceStart)

Enter minimum support (out of 100): 15
Enter minimum confidence (out of 100): 50
brute force timer start
db:  /content/drive/MyDrive/CS634/db1_transactions.json
{'google pixel 5 case'}  ->  {'usb c charger'} : [ 0.16666666666666666 ,  0.7142857142857142 ]
{'usb c charger'}  ->  {'google pixel 5 case'} : [ 0.16666666666666666 ,  0.8333333333333333 ]



db:  /content/drive/MyDrive/CS634/db2_transactions.json
{'sony wf-1000xm3'}  ->  {'sony wh-1000xm4'} : [ 0.16666666666666666 ,  1.0 ]
{'sony wh-1000xm4'}  ->  {'sony wf-1000xm3'} : [ 0.16666666666666666 ,  1.0 ]



db:  /content/drive/MyDrive/CS634/db3_transactions.json
{'microsoft surface'}  ->  {'rocketbook core'} : [ 0.23333333333333334 ,  1.0 ]
{'rocketbook core'}  ->  {'microsoft surface'} : [ 0.23333333333333334 ,  1.0 ]
{'lenovo x1c'}  ->  {'thunderbolt dongle'} : [ 0.23333333333333334 ,  1.0 ]
{'thunderbolt dongle'}  ->  {'lenovo x1c'} : [ 0.23333333333333334 ,  0.7777777777777778 ]



db:  /content/drive/MyDrive/CS634/db4_transac

In [ ]:
minSupport = float(input("Enter minimum support (out of 100): "))
minConfidence = float(input("Enter minimum confidence (out of 100): "))

print("apriori timer start")
print("\n\n")
timeAprioriStart = time.time()
for qq in ar:
  print("db: ", qq)
  apriori(qq, minSupport/100.0, minConfidence/100.0)
  print("\n\n")
timeAprioriEnd = time.time()
print("apriori time for all 5 databases: ", timeAprioriEnd - timeAprioriStart)

Enter minimum support (out of 100): 15
Enter minimum confidence (out of 100): 50
apriori timer start



db:  /content/drive/MyDrive/CS634/db1_transactions.json
('google pixel 5 case',)  ->  ('usb c charger',)  :  0.16666666666666666  , 0.7142857142857142
('usb c charger',)  ->  ('google pixel 5 case',)  :  0.16666666666666666  , 0.8333333333333333



db:  /content/drive/MyDrive/CS634/db2_transactions.json
('sony wf-1000xm3',)  ->  ('sony wh-1000xm4',)  :  0.16666666666666666  , 1.0
('sony wh-1000xm4',)  ->  ('sony wf-1000xm3',)  :  0.16666666666666666  , 1.0



db:  /content/drive/MyDrive/CS634/db3_transactions.json
('lenovo x1c',)  ->  ('thunderbolt dongle',)  :  0.23333333333333334  , 1.0
('microsoft surface',)  ->  ('rocketbook core',)  :  0.23333333333333334  , 1.0
('rocketbook core',)  ->  ('microsoft surface',)  :  0.23333333333333334  , 1.0
('thunderbolt dongle',)  ->  ('lenovo x1c',)  :  0.23333333333333334  , 0.7777777777777778



db:  /content/drive/MyDrive/CS634/db4_transact